In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv('../data/userData.csv')

df.drop(['weekday'], axis=1, inplace=True)
df

In [ ]:
from sklearn.preprocessing import LabelEncoder

data_encoded = pd.get_dummies(df[['time_of_day']]) # time of day is Night, Morning, Afternoon, Evening
data_encoded = pd.concat([df[['food_name']], data_encoded], axis=1) # food_name is a string 

y = df['food_name'].values

encoder = LabelEncoder()
encoder.fit(y)
encoded_y = encoder.transform(y)

data_encoded['encoded_y'] = encoded_y

data_encoded.sort_values('food_name', inplace=True)

data_encoded

In [ ]:
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [39]:
y = data_encoded['encoded_y'].values
X = data_encoded.drop(['food_name', 'encoded_y'], axis=1).values

print(tf.__version__)


# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=42)
X_train = X
y_train = y

num_classes = len(np.unique(y))
print(num_classes) # 139

model = tf.keras.Sequential([
    tf.keras.layers.Dense(32, activation='relu', input_shape=(4,), name='input'),
    tf.keras.layers.Dense(32, activation='relu', name='hidden1'),
    tf.keras.layers.Dense(32, activation='relu', name='hidden2'),
    tf.keras.layers.Dense(num_classes, activation='softmax', name='output')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

2.16.1
147


/usr/local/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
# model.evaluate(X_test, y_test)


In [40]:
# model.fit(X_train, y_train, batch_size=32, epochs=40, validation_data=(X_test, y_test))
model.fit(X_train, y_train, batch_size=32, epochs=30)

Epoch 1/30
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.0103 - loss: 4.9792    
Epoch 2/30
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0296 - loss: 4.9259     
Epoch 3/30
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0834 - loss: 4.8619 
Epoch 4/30
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2033 - loss: 4.7609 
Epoch 5/30
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1561 - loss: 4.6410 
Epoch 6/30
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2025 - loss: 4.4044 
Epoch 7/30
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1967 - loss: 4.1566 
Epoch 8/30
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2070 - loss: 3.8494 
Epoch 9/30
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1997 - loss: 3.8189  
Epoch 10/30
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1705 - loss: 3.7910 
Epoch 11/30
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2232 - loss: 3.6432 
Epoch 12/30
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - ac

In [ ]:
permutations_day = np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]], dtype=np.bool_)


predictions = model.predict(permutations_day)

top_n = 5

top_n_indices = np.argsort(predictions, axis=1)[:, -top_n:][:, ::-1]
top_n_probabilities = np.sort(predictions, axis=1)[:, -top_n:][:, ::-1]


top_n_indices, top_n_probabilities

flattened =  encoder.inverse_transform(top_n_indices.flatten())

top_n_labels = flattened.reshape(top_n_indices.shape)

top_n_labels, top_n_probabilities

In [ ]:
time_of_day_categories = ['Morning', 'Afternoon', 'Evening', 'Night']

with open('../out/time_based_output_tensorflow_network.txt', 'w') as f:
    # Iterate over time of day categories
    for time_of_day, predictions, probabilities in zip(time_of_day_categories, top_n_labels, top_n_probabilities):
        # Write time of day
        f.write("Time of Day: {}\n".format(time_of_day))
        f.write("Predictions:\n")
        # Iterate over predictions and probabilities
        for i, (prediction, probability) in enumerate(zip(predictions, probabilities), 1):
            # Write each prediction and its probability
            f.write("    - Prediction {}: {}, Probability: {:.2f}%\n".format(i, prediction, probability * 100))
        # Add a newline for better readability between time of day categories
        f.write("\n")
